# Estimation of PCA fitting for conversion from ZTE/PETRA to pseudo CT

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import Code
import sys
%matplotlib inline
from os.path import join as jn

# Step 1 - define inputs
* `BasePath`: Path to root directory where **T1W.nii.gz**, **ZTE.nii.gz** and **CT.nii.gz** files are located. **Be sure files are named exaclty like this**.
* `simbnibs_path` : Path to SimNIBS charm's output dir, it must be under BasePath
* `Mat4Trajectory` : Path to trajectory file exported from Brainsight. Be sure of calling **Target.txt**
* `ComputingDevice` : Type of processor in your computer, either M1, M2, M3 or M4 (no need to add max or pro monikers)
* `ID`' : ID for files, keep it with the value of **Target**
* `ZTE` : Path to ZTE or PETRA file, the file itself must be named **ZTE.nii.gz** regardless it is a ZTE or PETRA scan.
* `CTType` : Type of scan, use 2 for ZTE and 3 for PETRA

In [ ]:
BasePath='./DATA/ID_0068/'
simbnibs_path=jn(BasePath,'m2m_ID_0068')
Mat4Trajectory=jn(BasePath,'Target.txt')
ComputingDevice='M3'
ID = 'Target'

ZTE=jn(BasePath,'ZTE.nii.gz') #use always ZTE.nii.gz as file name regardless if ZTE o PETRA

CTType =2 # 2 = ZTE, 3 = PETRA ; CHANGE accordingly

# We instantiate the class
P=Code.Processing(BasePath,
                      ID,
                      simbnibs_path,
                      Mat4Trajectory,
                      ComputingDevice=ComputingDevice,
                      ZTE=ZTE,
                      CTType=CTType)

---
# Step 2: Run BabelBrain step 1 to generate files we will need for the processing
#### The code below will do the same process that is being called in Step 1 of the BabelBrain's GUI

In [ ]:
#REFERENCE
P.Run_mask_creation(bForceRecalculate=False)


---
# Step 3 - Run coregistration from CT to pseudo CT space (T1W space)

In [ ]:
P.RunElastix('FiniteDifferenceGradientDescent',bCToMRI=True)

----
#### Verify in fsleyes or other tool that the files `CT_in_T1W.nii.gz` and `ZTE_BiasCorrec_pCT.nii.gz` are correctly coregistered

# Step 4 - Run analysis of  pseudo CT with the default values
#### This will give us some initial assessment
**Important**: Adjust **LineSep** value to show only the upper part of the skull.

In [ ]:
P.ShowFirstPart(LineSep=120)

# Step 5 - Run PCA processing to find new fitting formula
#### Adjust ZTERange to be sure only the skull region is segmented


In [ ]:
P.ProcessMRI(ZTERange=[0.1,0.65])